নিম্নলিখিত নোটবুকগুলো চালানোর জন্য, যদি আপনি এখনও না করে থাকেন, তাহলে আপনাকে .env ফাইলে `OPENAI_API_KEY` হিসেবে ওপেনএআই কী সেট করতে হবে।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

পরবর্তী ধাপে, আমরা এম্বেডিং ইনডেক্সটি একটি প্যান্ডাস ডেটাফ্রেমে লোড করতে যাচ্ছি। এম্বেডিং ইনডেক্সটি `embedding_index_3m.json` নামক একটি JSON ফাইলে সংরক্ষিত আছে। এম্বেডিং ইনডেক্সে ২০২৩ সালের অক্টোবর মাসের শেষ পর্যন্ত প্রতিটি ইউটিউব ট্রান্সক্রিপ্টের জন্য এম্বেডিং সংরক্ষিত আছে।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

এবার আমরা `get_videos` নামে একটি ফাংশন তৈরি করব, যা Embedding Index-এ প্রশ্নের জন্য অনুসন্ধান করবে। এই ফাংশনটি প্রশ্নের সাথে সবচেয়ে বেশি মিল রয়েছে এমন শীর্ষ ৫টি ভিডিও ফেরত দেবে। ফাংশনটি নিচের ধাপগুলো অনুসরণ করে কাজ করে:

1. প্রথমে, Embedding Index-এর একটি কপি তৈরি করা হয়।
2. এরপর, প্রশ্নের Embedding OpenAI Embedding API ব্যবহার করে গণনা করা হয়।
3. তারপর Embedding Index-এ `similarity` নামে একটি নতুন কলাম তৈরি করা হয়। `similarity` কলামে প্রশ্নের Embedding এবং প্রতিটি ভিডিও সেগমেন্টের Embedding-এর মধ্যে কসাইন সাদৃশ্য (cosine similarity) রাখা হয়।
4. এরপর, Embedding Index-কে `similarity` কলাম অনুযায়ী ফিল্টার করা হয়। Embedding Index-এ শুধুমাত্র সেই ভিডিওগুলো রাখা হয় যেগুলোর কসাইন সাদৃশ্য ০.৭৫ বা তার বেশি।
5. সবশেষে, Embedding Index-কে `similarity` কলাম অনুযায়ী সাজানো হয় এবং শীর্ষ ৫টি ভিডিও ফেরত দেওয়া হয়।


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

এই ফাংশনটি খুবই সহজ, এটি শুধু অনুসন্ধান প্রশ্নের ফলাফলগুলি মুদ্রণ করে।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

১. প্রথমে, Embedding Index একটি Pandas Dataframe-এ লোড করা হয়।
২. এরপর, ব্যবহারকারীকে একটি কোয়েরি দিতে বলা হয়।
৩. তারপর `get_videos` ফাংশনটি ডাকা হয় কোয়েরি অনুসন্ধানের জন্য Embedding Index-এ।
৪. শেষে, `display_results` ফাংশনটি ডাকা হয় ব্যবহারকারীর কাছে ফলাফল দেখানোর জন্য।
৫. এরপর ব্যবহারকারীকে আবার একটি কোয়েরি দিতে বলা হয়। এই প্রক্রিয়া চলতে থাকে যতক্ষণ না ব্যবহারকারী `exit` লিখে।

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.bn.png)

আপনাকে একটি কোয়েরি দিতে বলা হবে। একটি কোয়েরি লিখে এন্টার চাপুন। অ্যাপ্লিকেশনটি কোয়েরির সাথে সম্পর্কিত ভিডিওগুলোর একটি তালিকা দেখাবে। এছাড়াও, ভিডিওর সেই অংশের একটি লিঙ্কও দেখাবে যেখানে প্রশ্নের উত্তরটি রয়েছে।

কিছু কোয়েরি উদাহরণ হিসেবে নিচে দেওয়া হলো:

- Azure Machine Learning কী?
- কনভল্যুশনাল নিউরাল নেটওয়ার্ক কীভাবে কাজ করে?
- নিউরাল নেটওয়ার্ক কী?
- আমি কি Azure Machine Learning-এর সাথে Jupyter Notebooks ব্যবহার করতে পারি?
- ONNX কী?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুল অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় রচিত নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
